<a href="https://colab.research.google.com/github/Edsonalex/class-planner/blob/main/Plano_de_Aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip -q install google-genai

In [53]:
# Configura a API Key do Google Gemini

import os
import google.generativeai as genai
from google.colab import userdata
import sys # Importa sys para sair do script

#Tentar obter a API Key dos "secrets" do Colab
api_key = os.getenv('GOOGLE_API_KEY')

# Se não encontrar nos segredos do Colab, tenta obter da variável de ambiente
if api_key is None:
    api_key = os.getenv('GOOGLE_API_KEY')
    print("INFO: Tentando obter a API Key da variável de ambiente GOOGLE_API_KEY.")


if api_key is not None:
    genai.configure(api_key=api_key)
    os.environ["GOOGLE_API_KEY"] = api_key
    print("API Key do Google Gemini configurada com sucesso.")
else:
    print("ERRO: A variável de ambiente ou segredo do Colab 'GOOGLE_API_KEY' não está configurada.")
    print("Por favor, configure sua chave da API do Google Gemini nos segredos do Colab ou como variável de ambiente.")
    sys.exit("API Key não encontrada. Saindo.")

API Key do Google Gemini configurada com sucesso.


In [54]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [55]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [56]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [57]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [58]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Planejador Pedagógico --- #
##########################################
class PlanejadorPedagogico:

    def __init__(self, model):
        self.agente_conteudo = AgenteConteudo(model)
        self.agente_atividades = AgenteAtividades(model)
        self.agente_avaliacao = AgenteAvaliacao(model)
        self.agente_agenda = AgenteAgenda(model)

    def planejar_aula(self, serie, disciplina, tema, duracao):
        conteudo = self.agente_conteudo.sugerir_conteudo(serie, disciplina, tema)
        atividades = self.agente_atividades.sugerir_atividades(serie, tema)
        avaliacoes = self.agente_avaliacao.sugerir_avaliacoes(tema)
        cronograma = self.agente_agenda.gerar_cronograma(duracao, conteudo, atividades, avaliacoes)

        return {
            "Série": serie,
            "Disciplina": disciplina,
            "Tema": tema,
            "Conteúdo": conteudo,
            "Atividades": atividades,
            "Avaliações": avaliacoes,
            "Cronograma": cronograma
        }

In [59]:
#########################################
# --- Agente 2: Criador de Conteúdo --- #
#########################################
class AgenteConteudo:
    def __init__(self, model):
        self.model = model
    def sugerir_conteudo(self, serie, disciplina, tema):
        prompt = (
            f"Você é um especialista em educação básica. Sugira um conteúdo didático para uma aula de {disciplina} "
            f"do {serie} ano do ensino fundamental sobre o tema '{tema}', de acordo com a BNCC. Seja objetivo e sucinto. Escreva como um professor."
        )
        response = self.model.generate_content(prompt)
        return response.text

In [60]:
###########################################
# --- Agente 3: Criador de Atividades --- #
###########################################
class AgenteAtividades:
    def __init__(self, model):
        self.model = model
    def sugerir_atividades(self, serie, tema):
        prompt = (
            f"Sugira duas atividades lúdicas e práticas para alunos do {serie} ano do ensino fundamental, "
            f"sobre o tema '{tema}'. As atividades devem ser simples e possíveis de realizar em sala de aula."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [61]:
###########################################
# --- Agente 4: Criador de Avaliações --- #
###########################################
class AgenteAvaliacao:
    def __init__(self, model):
        self.model = model
    def sugerir_avaliacoes(self, tema):
        prompt = (
            f"Sugira instrumentos de avaliação divididas nas cateorias Formativa (autoavaliação e observação) e Somativa(provas, testes, trabalhos) para o tema {tema} e a série {serie}.",
            f"Recomende avaliação escrita com problemas contextualizados."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [62]:
##########################################
# --- Agente 5: Criador de Agenda --- #
##########################################
class AgenteAgenda:
    def __init__(self, model):
      self.model = model
    def gerar_cronograma(self, duracao, conteudo, atividades, avaliacoes):
        prompt = (
            f"Organize os conteúdos, atividades e avaliações num cronograma semanal em formato de calendário considerando o tema{tema}, a duração {duracao}, as atividades propostas {atividades} e as avalaliações {avaliacoes}.",
            f"Considere número de 4 aulas por semana e o tempo de 50 minutos necessário por aula"
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [63]:
##############################
# --- Prompt e Resultado --- #
##############################
def iniciar_prompt():
    print("=== Gerador de Plano de Aula ===")
    serie = input("Série (ex: 5º): ")
    disciplina = input("Disciplina (ex: Matemática): ")
    tema = input("Tema (ex: Frações): ")
    duracao = int(input("Duração em semanas: "))
    return serie, disciplina, tema, duracao

import google.generativeai as genai

model = genai.GenerativeModel(MODEL_ID)
planejador = PlanejadorPedagogico(model)

serie, disciplina, tema, duracao = iniciar_prompt()

plano = planejador.planejar_aula(serie, disciplina, tema, duracao)

print("\n\n🚀 Iniciando o Sistema de Criação de Planejamento Pedagógico 🚀")
print(f"Maravilha! Vamos então criar o planejamento em {disciplina} para alunos da {serie} série com o tema {tema} e com duração de {duracao} semana(s).")

conteudo = plano["Conteúdo"]
print("\n--- 📝 Resultado do Gerador de Conteúdo (Conteúdo) ---\n")
display(to_markdown(conteudo))
print("--------------------------------------------------------------")

atividades = plano["Atividades"]
print("\n--- 📝 Resultado do Gerador de Atividades (Atividades) ---\n")
for atividade in atividades:
    display(to_markdown(atividade))
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

avaliacoes = plano["Avaliações"]
print("\n--- 📝 Resultado do Gerador de Avaliações (Avaliações) ---\n")
for avaliacao in avaliacoes:
    display(to_markdown(avaliacao))
print("--------------------------------------------------------------")

cronograma = plano["Cronograma"]
print("\n--- 📝 Resultado do Gerador de Cronograma (Cronograma) ---\n")
if isinstance(cronograma, list):
    for item in cronograma:
        display(to_markdown(item))
elif isinstance(cronograma, dict):
    for key, value in cronograma.items():
        display(to_markdown(f"{key}:\n{value}"))
else:
    display(to_markdown(str(cronograma)))

print("--------------------------------------------------------------")

=== Gerador de Plano de Aula ===
Série (ex: 5º): 1
Disciplina (ex: Matemática): reta numerica
Tema (ex: Frações): matematica
Duração em semanas: 1


🚀 Iniciando o Sistema de Criação de Planejamento Pedagógico 🚀
Maravilha! Vamos então criar o planejamento em reta numerica para alunos da 1 série com o tema matematica e com duração de 1 semana(s).

--- 📝 Resultado do Gerador de Conteúdo (Conteúdo) ---



> Olá, pequenos exploradores da matemática!
> 
> Hoje vamos desvendar a RETA NUMÉRICA, uma ferramenta super útil para entendermos os números e suas posições.
> 
> **Objetivo:** Compreender a sequência numérica (0 a 10) e sua representação na reta numérica. (Alinhado com a BNCC: EF01MA01 - Utilizar números naturais como indicador de quantidade ou de ordem em diferentes situações cotidianas e reconhecer situações em que os números não indicam contagem nem ordenação, mas sim código de identificação.)
> 
> **Materiais:**
> 
> *   Reta numérica grande (desenhada no quadro ou impressa).
> *   Fichas com os números de 0 a 10.
> *   Objetos pequenos (ex: lápis, borracha, tampinhas).
> 
> **Desenvolvimento:**
> 
> 1.  **Exploração Inicial:** Apresente a reta numérica e explique que ela representa uma sequência de números, como uma "estrada" onde cada número tem seu lugar.
> 2.  **Colocando os Números:** Distribua as fichas com os números e peça para as crianças, uma a uma, colarem no lugar correto na reta numérica. Incentive a contagem e a comparação ("Qual número vem antes do 3?").
> 3.  **Brincando na Reta:**
>     *   **"Onde estou?":** Coloque um objeto em um número na reta e peça para as crianças dizerem qual número é.
>     *   **"Andando na Reta":** Dê comandos simples: "Comece no 2 e avance 3 casas. Em qual número você está agora?". Use objetos para representar o movimento.
> 4.  **Registro:** Desenhe uma reta numérica simples no caderno e peça para as crianças escreverem os números de 0 a 10.
> 
> **Avaliação:** Observe a participação das crianças nas atividades e a capacidade de identificar e ordenar os números na reta.
> 
> **Dica:** Use cores diferentes para cada número na reta numérica para facilitar a identificação. Transforme a aula em uma grande brincadeira!

--------------------------------------------------------------

--- 📝 Resultado do Gerador de Atividades (Atividades) ---



> ## Sugestões de Atividades Lúdicas e Práticas de Matemática para o 1º Ano:

> **Foco:** Números, Contagem, Formas Geométricas, Tamanho e Comparação.

...

> *   Registre as observações em um diário de bordo para acompanhar o progresso individual.

> *   Ao final da semana, revise os principais conceitos abordados.

> *   A avaliação formativa é contínua e deve ser utilizada para ajustar as estratégias de ensino.

> Este cronograma oferece uma estrutura flexível que pode ser adaptada de acordo com as necessidades e o ritmo de aprendizado da turma. Lembre-se que a ludicidade e a participação ativa dos alunos são fundamentais para o sucesso no aprendizado da matemática no 1º ano.

--------------------------------------------------------------
